based on
https://github.com/lilianweng/stock-rnn/blob/master/main.py
https://lilianweng.github.io/lil-log/2017/07/08/predict-stock-prices-using-RNN-part-1.html
using data from
https://finance.yahoo.com/quote/%5EGSPC/history?period1=-631015200&period2=1515765600&interval=1d&filter=history&frequency=1d

scp -i /c/Users/bmt/.ssh/aws-key.pem /d/2017_working/yahoo_finance_data_GSPC.csv ubuntu@ec2-54-187-235-164.us-west-2.compute.amazonaws.com://home/ubuntu/fastai/data/RNN_predict_stock_prices/

In [3]:
import os
import pandas as pd
import pprint

import tensorflow as tf
import tensorflow.contrib.slim as slim


In [15]:
BASE_PATH = "/home/ubuntu/fastai_bmt_reworking/stock_prediction/"
LOGS_PATH = "/home/ubuntu/fastai/data/RNN_predict_stock_prices/logs/"
DATA_PATH = "/home/ubuntu/fastai/data/RNN_predict_stock_prices/"

In [6]:
os.chdir(BASE_PATH)
from data_model import StockDataSet
from model_rnn import LstmRNN

In [7]:
flags = tf.app.flags
flags.DEFINE_integer("stock_count", 100, "Stock count [100]")
flags.DEFINE_integer("input_size", 1, "Input size [1]")
flags.DEFINE_integer("num_steps", 30, "Num of steps [30]")
flags.DEFINE_integer("num_layers", 1, "Num of layer [1]")
flags.DEFINE_integer("lstm_size", 128, "Size of one LSTM cell [128]")
flags.DEFINE_integer("batch_size", 64, "The size of batch images [64]")
flags.DEFINE_float("keep_prob", 0.8, "Keep probability of dropout layer. [0.8]")
flags.DEFINE_float("init_learning_rate", 0.001, "Initial learning rate at early stage. [0.001]")
flags.DEFINE_float("learning_rate_decay", 0.99, "Decay rate of learning rate. [0.99]")
flags.DEFINE_integer("init_epoch", 5, "Num. of epoches considered as early stage. [5]")
flags.DEFINE_integer("max_epoch", 50, "Total training epoches. [50]")
flags.DEFINE_integer("embed_size", None, "If provided, use embedding vector of this size. [None]")
flags.DEFINE_string("stock_symbol", None, "Target stock symbol [None]")
flags.DEFINE_integer("sample_size", 4, "Number of stocks to plot during training. [4]")
flags.DEFINE_boolean("train", False, "True for training, False for testing [False]")

FLAGS = flags.FLAGS

In [8]:
pp = pprint.PrettyPrinter()


In [12]:
if not os.path.exists(LOGS_PATH):
    os.mkdir(LOGS_PATH)

In [13]:
def show_all_variables():
    model_vars = tf.trainable_variables()
    slim.model_analyzer.analyze_vars(model_vars, print_info=True)

In [16]:
def load_sp500(input_size, num_steps, k=None, target_symbol=None, test_ratio=0.05):
    if target_symbol is not None:
        return [
            StockDataSet(
                target_symbol,
                input_size=input_size,
                num_steps=num_steps,
                test_ratio=test_ratio)
        ]

    # Load metadata of s & p 500 stocks
    info = pd.read_csv(DATA_PATH + "yahoo_finance_data_GSPC.csv")
    info = info.rename(columns={col: col.lower().replace(' ', '_') for col in info.columns})
    info['file_exists'] = info['symbol'].map(lambda x: os.path.exists("data/{}.csv".format(x)))
    print info['file_exists'].value_counts().to_dict()

    info = info[info['file_exists'] == True].reset_index(drop=True)
    info = info.sort('market_cap', ascending=False).reset_index(drop=True)

    if k is not None:
        info = info.head(k)

    print "Head of S&P 500 info:\n", info.head()

    # Generate embedding meta file
    info[['symbol', 'sector']].to_csv(os.path.join("logs/metadata.tsv"), sep='\t', index=False)

    return [
        StockDataSet(row['symbol'],
                     input_size=input_size,
                     num_steps=num_steps,
                     test_ratio=0.05)
        for _, row in info.iterrows()]

In [17]:
pp.pprint(flags.FLAGS.__flags)

# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
run_config = tf.ConfigProto()
run_config.gpu_options.allow_growth = True


{}


In [18]:
with tf.Session(config=run_config) as sess:
    rnn_model = LstmRNN(
        sess,
        FLAGS.stock_count,
        lstm_size=FLAGS.lstm_size,
        num_layers=FLAGS.num_layers,
        num_steps=FLAGS.num_steps,
        input_size=FLAGS.input_size,
        keep_prob=FLAGS.keep_prob,
        embed_size=FLAGS.embed_size,
    )

    show_all_variables()

    stock_data_list = load_sp500(
        FLAGS.input_size,
        FLAGS.num_steps,
        k=FLAGS.stock_count,
        target_symbol=FLAGS.stock_symbol,
    )

    if FLAGS.train:
        rnn_model.train(stock_data_list, FLAGS)
    else:
        if not rnn_model.load()[0]:
            raise Exception("[!] Train a model first, then run test mode")


/usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


---------
Variables: name (type shape) [size]
---------
dynamic_rnn/lstm_cell/kernel:0 (float32_ref 129x512) [66048, bytes: 264192]
dynamic_rnn/lstm_cell/bias:0 (float32_ref 512) [512, bytes: 2048]
w:0 (float32_ref 128x1) [128, bytes: 512]
b:0 (float32_ref 1) [1, bytes: 4]
Total size of variables: 66689
Total bytes of variables: 266756


KeyError: 'symbol'